In [2]:
import pygame
import numpy as np
import matplotlib.pyplot as plt

pygame 2.5.2 (SDL 2.28.3, Python 3.12.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


### Boid Flocking

In [3]:
class Boid:
    def __init__(self, position, velocity, perception):
        self.position = position
        self.velocity = velocity
        self.perception = perception

    def flock(self, boids):
        center_of_vel = np.zeros(2)
        center_of_sep = np.zeros(2)
        center_of_pos = np.zeros(2)

        counter = 0
        for other in boids:
            if other != self:
                distance = np.linalg.norm(other.position - self.position)
                if distance < self.perception:
                    center_of_vel += other.velocity
                    center_of_sep += (self.position - other.position) / np.linalg.norm(self.position - other.position)**2
                    center_of_pos += other.position
                    counter += 1

        if counter > 0:
            center_of_vel = center_of_vel / counter
            center_of_sep = center_of_sep / counter
            center_of_pos = center_of_pos / counter

            alignment_factor = 0.01
            separation_factor = 0.5
            cohesion_factor = 0.001
            
            alignment = center_of_vel * alignment_factor
            separation = center_of_sep * separation_factor
            cohesion = (center_of_pos - self.position) * cohesion_factor
            self.velocity = self.velocity + alignment + separation + cohesion

        max_speed = 3
        self.velocity = self.velocity / np.linalg.norm(self.velocity) * max_speed
        self.position += self.velocity
        

    def draw(self, screen):
        size = 4
        color = (0, 0, 0)
        pygame.draw.circle(screen, color, self.position, size)
        

In [6]:
pygame.init()

width = 400
height = 400
ticks = 2000
screen = pygame.display.set_mode((width, height))
clock  = pygame.time.Clock()
font = pygame.font.Font(None, 24)
data = []

# initialize game 
boids = []
for i in range(30):
    position = np.random.uniform(0, 400, 2)
    velocity = np.random.uniform(-1, 1, 2)
    perception = 100
    boid = Boid(position, velocity, perception)
    boids.append(boid)

running = True
for tick in range(ticks):

    # set the frames per second FPS
    clock.tick(60)

    # set the initial screen color in RGB
    screen.fill((255, 255, 255))

    # update game
    for boid in boids:
        boid.flock(boids)

    # handle borders
    for boid in boids:
        x, y = boid.position
        if x < 0:
            boid.position[0] = 400
        if x > 400:
            boid.position[0] = 0
        if y < 0:
            boid.position[1] = 400
        if y > 400:
            boid.position[1] = 0
            
        
    # draw update
    for boid in boids:
        boid.draw(screen)   

    # display and update the current tick
    text = font.render(str(tick), True, (0, 0, 0))
    screen.blit(text, (10, 10))
    
    # update the whole screen
    pygame.display.flip()

    # store data for each tick
    total_cosine_similarity = 0
    average_velocity = np.mean([boid.velocity for boid in boids], axis=0)

    for boid in boids:
        cosine_similarity = np.dot(boid.velocity, average_velocity) / (np.linalg.norm(boid.velocity) * np.linalg.norm(average_velocity))
        total_cosine_similarity += cosine_similarity

    total_cosine_similarity = total_cosine_similarity / len(boids)
    data.append(total_cosine_similarity)

    # handle the exit buttom
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            running = False
            
    # and break the loop
    if not running:
        break
    
pygame.quit()

In [7]:
print(data)

[0.26036718189584757, 0.2605674221641922, 0.260380146947466, 0.2594514526717023, 0.25863273940212184, 0.2587160090101609, 0.25869739368613187, 0.2580307566138244, 0.2573150044807654, 0.2573381279565122]
